In [108]:
from stanfordcorenlp import StanfordCoreNLP
import logging
import json
import pandas as pd
# from nltk.parse.stanford import StanfordDependencyParser
import os
import numpy as np
from graphviz import Source


In [109]:
class StanfordNLP:
    def __init__(self, host='http://localhost', port=9000):
        self.nlp = StanfordCoreNLP(host, port=port,
                                   timeout=30000)  # , quiet=False, logging_level=logging.DEBUG)
        self.props = {
            'annotators': 'tokenize,ssplit,pos,lemma,ner,parse,depparse,dcoref,relation',
            'pipelineLanguage': 'en',
            'outputFormat': 'json'
        }

    def word_tokenize(self, sentence):
        return self.nlp.word_tokenize(sentence)

    def pos(self, sentence):
        return self.nlp.pos_tag(sentence)

    def ner(self, sentence):
        return self.nlp.ner(sentence)

    def parse(self, sentence):
        return self.nlp.parse(sentence)

    def dependency_parse(self, sentence):
        return self.nlp.dependency_parse(sentence)

    def annotate(self, sentence):
        return json.loads(self.nlp.annotate(sentence, properties=self.props))

    @staticmethod
    def tokens_to_dict(_tokens):
        tokens = defaultdict(dict)
        for token in _tokens:
            tokens[int(token['index'])] = {
                'word': token['word'],
                'lemma': token['lemma'],
                'pos': token['pos'],
                'ner': token['ner']
            }
        return tokens

In [110]:
#allow full data view
pd.set_option('display.max_colwidth', -1)

In [111]:
event_df = pd.read_csv('df.csv')
# df['event_summary']=df.event_summary.astype(str)


In [112]:
event_df=event_df.drop("event_id",axis=1)

In [113]:
event_df.loc[:,'event_id'] = np.arange(len(event_df))

In [114]:
event_df=event_df[['event_id','date','category','event_title','event_summary']]

In [115]:
event_df=event_df[0:2000]

In [116]:
event_df.loc[10,'event_summary']='The Islamic State of Iraq and Levant claims responsibility for a suicide bomb attack on a Iraqi Army base Camp Speicher now known as Tikrit Air Academy near town of Tikrit north of Baghdad that killed at least fifteen and injured twentytwo'
event_df.loc[21,'event_summary']='The Islamic State of Iraq and Levants branch in Libya attacks Libyan oil Port of sidra which is Libyas largest oil depot. Two members of Petrol Facilities Guard militia which is defending Sidra were reportedly killed in clashes'
event_df.loc[22,'event_summary']='Trading on Shanghai Stock Exchange and Shenzhen Stock Exchange are suspended for day after stock indices plummeted by six. In US Dow Jones industrial average closes 275 points down due to concerns over Chinese economy'
event_df.loc[36,'event_summary']='Islamic State militants continue to besiege Libyan oil port of Sidra attacking checkpoints on road to Port leaving seven guards dead. The National Oil Corporation NOC said an oil storage tank was set on fire by a longrange rocket and that oil tank fire started just as firefighters were close to bringing under control another blaze at an oil tank that was hit during fighting in nearby Ras Lanuf Refinery on Monday'
event_df.loc[39,'event_summary']='Backtobacktoback storms from current El NiAo which National Oceanic and Atmospheric Administration says ties strongest on record 1997 98 are likely to produce dangerous flooding and mudslides in Western US with as much as fifteen inches thirty cm of rain and about two feet sixty cm of snow in Sierra Nevada mountains over next two weeks'
event_df.loc[45,'event_summary']='The Islamic States Libyan branch continues attacks on Libyas oil ports of Sidra and Ras Lanuf resulting in at least ten security personnel deaths and fourty injured'
event_df.loc[89,'event_summary']='The newly appointed President of Catalonia Carles Puigdemont vows to continue his predecessor Artur Mas GavarrAs plans to secede from Spain within eighteen months'
event_df.loc[97,'event_summary']='In Australian football thirtyfive current and former players with Essendon Football Club are banned from playing in Australian Football League during 2016 AFL season after Court of Arbitration for Sport finds that they took banned substance thymosinbeta four'


In [117]:
sNLP = StanfordNLP()

In [118]:
from nltk.parse.corenlp import CoreNLPParser
from nltk.parse.stanford import StanfordDependencyParser
from nltk.tree import Tree


In [119]:
Args_df = pd.DataFrame()
Args_df['Arg_ID']='NA'
Args_df['event_id']='NA'
Args_df['argtext']='NA'
Args_df['same']='NA'

In [121]:
ann = StanfordNLP()
ann.props = {
            'annotators': 'ssplit',
            'pipelineLanguage': 'en',
            'outputFormat': 'json'
        }

l=0

for j in range(0,2000):
    xx=0
    fo = open('verbphrase', 'w', encoding="utf-8")
    text = event_df.loc[j,'event_summary']
    id1 = event_df.loc[j,'event_id']
    print(id1)
    print("\n")
    print(text+"\n")
    sentences = ann.annotate(text)
    for sentence in sentences['sentences']:
        sent_str = ''.join([key_word['before'] + key_word['word'] for key_word in sentence['tokens']]).strip()
#         print (sent_str)
        subs_ = set()
        parsestr=sNLP.parse(sent_str)
#         print (parsestr)
        root_tree = Tree.fromstring(parsestr)
        parsed_arguments = {}
        for  i in root_tree.subtrees():
            if i.label() == 'NP' and str(i) not in subs_:
                np_subs = [str(j) for j in i.subtrees() if j.label() in ['NP','VP','PP'] and j != i]
                
                count = len(np_subs)
                if count <= 1:
                    subs_.update(np_subs)
                    subs_.add(str(i))
                
                if count > 1:
                    continue
 
                fo.write(str( i.leaves())+ ';')
                print(str( i.leaves()))

    fo.close()        
    with open('verbphrase', 'r',encoding="utf8") as file:
        data = file.read().replace('\n', '')

        split_data=data.split(';')
#         print (split_data[0])
#         print(len(split_data))
        
        for k in range(0,len(split_data)):
#             k=k+1
#             df['Nounphrases'+ str(k) ]="random"
#             print(l)
            Args_df.loc[l,'argtext']=split_data[k]
            Args_df.loc[l,'event_id']=event_df.loc[j,'event_id']
            Args_df.loc[l,'same']=xx
            Args_df.loc[l,'argtext'] = str(Args_df.loc[l,'argtext']).strip('[]')
            Args_df.loc[l,'argtext'] = str(Args_df.loc[l,'argtext']).replace(',','')
            Args_df.loc[l,'argtext'] = str(Args_df.loc[l,'argtext']).replace('\'','')
            Args_df.loc[l,'argtext'] = str(Args_df.loc[l,'argtext']).replace('\"','')
            xx=xx+1
            l=l+1
        file.seek(0)
    fo = open('verbphrase', 'w').close()


0


A shooting takes place at a pub in Tel Aviv Israel leaving two dead and eight injured. The gunman managed to escape and possibly killed his accomplice an Arab taxi driver whose body was found nearby. The police identified shooter as 30 year old muslim Nisaat Melkham from Arara and searched his home

['A', 'shooting']
['place']
['a', 'pub']
['Tel', 'Aviv', 'Israel']
['two']
['eight']
['The', 'gunman']
['his', 'accomplice']
['an', 'Arab', 'taxi', 'driver']
['The', 'police']
['shooter']
['30', 'year', 'old', 'muslim', 'Nisaat', 'Melkham']
['Arara']
['his', 'home']
1


A new 28foot tall statue of Jesus dubbed Jesus de Greatest is unveiled on New Years Day outside St. Aloysius Catholic Church in Abajah village Nigerias Imo state which is described as tallest Jesus statue in Africa

['A', 'new', '28foot', 'tall', 'statue']
['Jesus']
['Jesus', 'de', 'Greatest']
['New', 'Years']
['St.', 'Aloysius', 'Catholic', 'Church']
['Abajah', 'village', 'Nigerias', 'Imo']
['tallest', 'Jesus', 'statue'

['Two', 'Sunni', 'mosques']
['central', 'Iraq']
['a', 'muezzin']
['fears']
['renewals']
['sectarian', 'strife']
['Saudi', 'Arabias', 'execution']
['Shiite', 'cleric', 'Nimr', 'alNimr']
['Saudi', 'Arabia']
19


The Islamic State of Iraq and Levant releases a video showing killing of five men accused of committing espionage on militant group for United Kingdom. The British Foreign and Commonwealth Office released a statement saying that they are examining content of Video

['The', 'Islamic', 'State']
['Iraq', 'and', 'Levant']
['a', 'video']
['killing']
['five', 'men']
['espionage']
['militant', 'group']
['United', 'Kingdom']
['The', 'British', 'Foreign', 'and', 'Commonwealth', 'Office']
['a', 'statement']
['they']
['content']
['Video']
20


Five of terrorists who attacked an Indian Air Force base in Pathankot Punjab are reported to have been killed moppingup and search operations continue. National Security Guard officials state Operations will continue until air base is made safe

['Fiv

['A', 'bus']
['fire']
['Chinese', 'city']
['Yinchuan']
['at', 'least', 'fourteen', 'deaths']
['30']
39


Backtobacktoback storms from current El NiAo which National Oceanic and Atmospheric Administration says ties strongest on record 1997 98 are likely to produce dangerous flooding and mudslides in Western US with as much as fifteen inches thirty cm of rain and about two feet sixty cm of snow in Sierra Nevada mountains over next two weeks

['Backtobacktoback', 'storms']
['current', 'El', 'NiAo']
['National', 'Oceanic', 'and', 'Atmospheric', 'Administration']
['ties']
['strongest']
['record']
['1997', '98']
['dangerous', 'flooding', 'and', 'mudslides']
['Western', 'US']
['as', 'much', 'as', 'fifteen', 'inches']
['thirty', 'cm']
['rain']
['two', 'feet']
['sixty', 'cm']
['snow']
['Sierra', 'Nevada', 'mountains']
['next', 'two', 'weeks']
40


Saudi Arabian Foreign Minister Adel alJubeir says countrys recent tensions with Iran will not interfere with Syrian peace talks scheduled to begin in

['Trade']
['Shanghai', 'Stock', 'Exchange']
['Shenzhen', 'Stock', 'Exchange']
['a', 'second', 'time']
['a', 'week']
['share', 'prices']
['following', 'devaluation']
['yuan']
['The', 'Chinese', 'securities', 'regulator']
['legislation']
['sale']
['shares']
['shareholders']
['The', 'Dow', 'Jones', 'Industrial', 'Average']
['just', 'under', '400', 'points']
['a', 'result']
['concerns']
['Chinese', 'economy']
59


In US a swarm of 30 and counting earthquakes of magnitude 2.5 or greater struck Oklahoma from Wednesday evening through Thursday midafternoon. The biggest a magnitude 4.8 happened yesterday at 10:27 pm. CST 32 kms 20 miles northwest of Fairview Oklahoma

['US']
['a', 'swarm']
['30']
['counting', 'earthquakes']
['magnitude', '2.5', 'or', 'greater']
['Oklahoma']
['Wednesday']
['Thursday', 'midafternoon']
['The', 'biggest']
['a', 'magnitude', '4.8']
['10:27', 'pm']
['CST']
['32', 'kms']
['20', 'miles']
['Fairview', 'Oklahoma']
60


South Korea says it will resume crossborder propaga


['Twenty', 'people']
['a', 'bus']
['amateur', 'football', 'players', 'and', 'fans']
['a', 'river']
['Veracruz', 'eastern']
['Mexico']
84


The two monthold Nipigon River Bridge near Nipigon Ontario is closed to traffic after a mechanical failure severing TransCanada Highway and forcing a detour into US

['The', 'two', 'monthold']
['Nipigon', 'River', 'Bridge']
['Nipigon', 'Ontario']
['traffic']
['a', 'mechanical', 'failure']
['TransCanada', 'Highway']
['a', 'detour']
['US']
85


A single US Air Force B52 longrange strategic bomber makes a lowlevel flight over South Korea close to Korean Demilitarized Zone DMZ in a show of force following Northkoreas fourth nuclear test on January six before returning to its base in Guam

['A', 'single', 'US', 'Air', 'Force', 'B52', 'longrange']
['strategic', 'bomber']
['a', 'lowlevel', 'flight']
['South', 'Korea']
['Korean', 'Demilitarized', 'Zone', 'DMZ']
['a', 'show']
['force']
['Northkoreas', 'fourth', 'nuclear', 'test']
['January', 'six']
['its', 

...

['The', 'protests']
['a', 'day']
['maduro']
['oppositioncontrolled', 'National', 'Assembly']
1886


Syrian government forces and allies including Lebanese Hezbollah fighters retake a large area southeast of Damascus including key town of Deir alAsafir. Fighting between Jaysh alIslam and rival factions led by alQaeda Nusra Front in Ghouta near damascus leaves more than fifty people dead

['Syrian', 'government', 'forces', 'and', 'allies']
['Lebanese', 'Hezbollah', 'fighters']
['a', 'large', 'area', 'southeast']
['Damascus']
['key', 'town']
['Deir', 'alAsafir']
['Fighting']
['Jaysh', 'alIslam', 'and', 'rival', 'factions']
['alQaeda', 'Nusra', 'Front']
['Ghouta']
['damascus']
['more', 'than', 'fifty', 'people']
1887


Eight policemen are killed in an insider attack in Zabul Province

['Eight', 'policemen']
['an', 'insider', 'attack']
['Zabul', 'Province']
1888


At least three civilians are killed and ten others wounded in a bomb attack in Baghdad Iraq

['At', 'least', 'three', 'civilians

['Tsai', 'Ingwen']
['first', 'female', 'President']
['Taiwan']
1912


Israeli Defense minister Moshe Yaalon announces his resignation

['Israeli', 'Defense', 'minister', 'Moshe', 'Yaalon']
['his', 'resignation']
1913


The Oklahoma legislature passes a bill that would make it a felony for doctors to perform abortions and revokes medical license of assisting physicians

['The', 'Oklahoma', 'legislature']
['a', 'bill']
['it']
['a', 'felony']
['doctors']
['abortions']
['medical', 'license']
['physicians']
1914


Iraqi security forces fire tear gas and water cannon on protesters attempting to storm Green Zone in Baghdad injuring at least 33

['Iraqi', 'security', 'forces']
['fire']
['gas', 'and', 'water', 'cannon']
['protesters']
['Green', 'Zone']
['Baghdad']
['at', 'least', '33']
1915


A homeopathic doctor is hacked to death in Bangladesh. The Islamic State of Iraq and Levant claim responsibility

['A', 'homeopathic', 'doctor']
['death']
['Bangladesh']
['The', 'Islamic', 'State']
['Iraq'

1942


 reports nearly 150 people are killed and at least 200 wounded in a series of car bomb and suicide attacks in Syrian cities of Jableh and Tartus in governmentcontrolled territory that hosts Russian military bases. The Islamic State of Iraq and Levant group claims responsibility

['reports']
['150', 'people']
['a', 'series']
['car', 'bomb', 'and', 'suicide', 'attacks']
['Syrian', 'cities']
['Jableh', 'and', 'Tartus']
['territory']
['Russian', 'military', 'bases']
['The', 'Islamic', 'State']
['Iraq']
['Levant', 'group']
['responsibility']
1943


At least 41 army recruits are killed and 60 others injured in a suicide car bomb attack in Aden Yemen

['At', 'least', '41', 'army', 'recruits']
['and', '60', 'others']
['a', 'suicide', 'car', 'bomb', 'attack']
['Aden', 'Yemen']
1944


The Iraqi government begins Operation Breaking Terrorism aimed at retaking city of Fallujah which has been under control of Islamic State of Iraq and Levant group since early 2014

['The', 'Iraqi', 'governme

1969


A court in US state of Pennsylvania finds that there is enough evidence to hold entertainer Bill Cosby on felony indecent assault charges in relation to case of Andrea Constand v. William H. Cosby Jr.

['A', 'court']
['US', 'state']
['Pennsylvania']
['there']
['entertainer', 'Bill', 'Cosby']
['felony', 'indecent', 'assault', 'charges']
['relation']
['case']
['Andrea', 'Constand', 'v.', 'William', 'H.', 'Cosby', 'Jr.']
1970


Googles offices in central Paris are raided by French finance officials as part of a tax fraud investigation. Google is accused of owing 1.6 bn dollar 1.8 bn Pound 1.3 bn in unpaid taxes

['Googles', 'offices']
['central', 'Paris']
['French', 'finance', 'officials']
['part']
['a', 'tax', 'fraud', 'investigation']
['Google']
['owing', '1.6', 'bn', 'dollar', '1.8', 'bn', 'Pound']
['1.3', 'bn']
['unpaid', 'taxes']
1971


Loretta Lynch Attorney General of US advises that US government will seek death penalty against Dylann Roof accused of conducting 2015 Charles

['google']
['copyrighted', 'code']
['Android']
['operating', 'system']
['googles']
1993


A bill that punishes hate crimes against law enforcement is signed by Louisiana Governor John Bel Edwards. It is first state to pass such a law

['A', 'bill']
['hate', 'crimes']
['law', 'enforcement']
['Louisiana', 'Governor', 'John', 'Bel', 'Edwards']
['It']
['such', 'a', 'law']
1994


A Muslim mob is found to have attacked Coptic homes and stripped a woman in public in Minya Governorate Egypt on 20 May 2016

['A', 'Muslim', 'mob']
['Coptic', 'homes']
['a', 'woman']
['public']
['Minya', 'Governorate', 'Egypt']
['20']
1995


French labor unions led by CGT implement a strike in protest of proposed reforms to labor laws

['French', 'labor', 'unions']
['CGT']
['a', 'strike']
['protest']
['proposed', 'reforms']
['labor', 'laws']
1996


The Associated Press reports that Donald Trump has secured enough delegates to secure Republican Party nomination for President

['The', 'Associated', 'Press']
['Donald

In [122]:
import nltk
import re
from pycorenlp import *
nlp = StanfordCoreNLP("http://localhost:9000/")
event_df['Entity Relation']=np.nan

In [123]:
for k in range(0,2000):
    text = event_df.loc[k,'event_summary']
    print('sentence:'+text+ '\n')
    output = nlp.annotate(text, properties={"annotators":"tokenize,ssplit,pos,depparse,natlog,openie",
                            "outputFormat": "json",
                             "openie.triple.strict":"true",
                             "openie.max_entailments_per_clause":"1"})
    result = [output["sentences"][0]["openie"] for item in output]
    for i in result:
        for rel in i:
            relationSent=rel['subject'],rel['relation'],rel['object']
            print(relationSent )
            print('\n')
            event_df['Entity Relation'][k]= relationSent

sentence:A shooting takes place at a pub in Tel Aviv Israel leaving two dead and eight injured. The gunman managed to escape and possibly killed his accomplice an Arab taxi driver whose body was found nearby. The police identified shooter as 30 year old muslim Nisaat Melkham from Arara and searched his home

('pub', 'is in', 'Tel Aviv Israel')


sentence:A new 28foot tall statue of Jesus dubbed Jesus de Greatest is unveiled on New Years Day outside St. Aloysius Catholic Church in Abajah village Nigerias Imo state which is described as tallest Jesus statue in Africa



c:\users\gulja\python 3.7.3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


('Jesus de Greatest', 'is', 'unveiled on New Years Day outside St. Aloysius Catholic Church')


('tallest Jesus statue', 'is in', 'Africa')


sentence:About one thousand houses in Manilas Tondo district in Philippines are set ablaze following New Years Eve firecracker festivities that left person dead and 380 others injured

('one thousand houses', 'is in', 'Manilas Tondo district')


sentence:The EUUkraine Free Trade deal officially comes into force coinciding with a Russian food embargo on Ukraine

('EUUkraine Free Trade', 'deal officially', 'comes into force coinciding with Russian food embargo on Ukraine')


sentence:The twochild policy takes effect in China allowing couples in country to have at most two children replacing controversial onechild policy. The change in law was announced by ruling Communist Party on October 29 and passed Standing Committee of National Peoples Congress on December 27 five days prior to its effect

('effect', 'allowing', 'couples in country')


('coupl

('US Army soldier two others', 'wounded in', 'joint operation Afghan National Security Forces Afghanistans southern Helmand Province')


('joint operation', 'is in', 'Afghanistans southern Helmand Province')


('joint operation', 'is with', 'Afghan National Security Forces')


sentence:Violent riots break out in Kaliachak West Bengal India after political activist Kamlesh Tiwari allegedly insulted Muslim prophet Muhammad

sentence:One Shia protester is killed and a child is injured by Saudi police in AlAwamiyah Eastern Province

('child', 'is injured by', 'Saudi police in AlAwamiyah Eastern Province')


('Saudi police', 'is in', 'AlAwamiyah Eastern Province')


sentence:Islamic State militants continue to besiege Libyan oil port of Sidra attacking checkpoints on road to Port leaving seven guards dead. The National Oil Corporation NOC said an oil storage tank was set on fire by a longrange rocket and that oil tank fire started just as firefighters were close to bringing under control an

sentence:Three people are unaccounted for and at least 135 structures are destroyed including historic yarloop Workshops following a bushfire in small Western Australian town of Yarloop

('bushfire', 'is in', 'small Western Australian town of Yarloop')


sentence:The chief of Colognes police force Wolfgang Albers is relieved of his duties following criticism of his handling of violent clashes and sexual assaults by migrant gangs in city on New Years Eve

('migrant gangs', 'is in', 'city')


sentence:The Mexican Navy recaptures escaped drug lord JoaquAn GuzmAin also known as El Chapo in Los Mochis Sinaloa

('Mexican Navy', 'recaptures', 'escaped drug lord JoaquAn GuzmAin also known as El Chapo in Los Mochis Sinaloa')


('El Chapo', 'is in', 'Los Mochis Sinaloa')


sentence:Dragan VasiljkoviA a former Serbian paramilitary commander is charged with war crimes in Croatia

('war crimes', 'is in', 'Croatia')


('former Serbian paramilitary commander', 'is charged with', 'war crimes in Croati

...

('recognizing', 'civil unions of', 'samesex couples')


sentence:Uganda arrests opposition leader Kizza Besigye as he addressed a crowd of supporters in capital Kampala and shuts down access to social media within country prior to President Yoweri Musevenis inauguration tomorrow

('crowd', 'is in', 'capital Kampala')


sentence:Al Qaeda militants kill at least thirteen Yemeni government troops near city of Mukalla

('Al Qaeda militants', 'kill', 'at least thirteen Yemeni government troops')


sentence:A twin suicide bombing attack at a police station in Baghdad kills five people. Yesterday 93 people had been killed and 165 injured in deadliest day of attacks in Iraq this year

('police station', 'is in', 'Baghdad')


sentence:Nissan announces that it will buy 34 percent of fellow Japanese car manufacturer Mitsubishi Motors for 237.4 billion yen dollar 2.2 billion

sentence:The Bank of England issues an unusually sharp warning about possible economic consequences of an Out vote in upcom

('interview', 'is with', 'former US diplomat Donald Rickard')


sentence:Voters in Dominican Republic go to polls for a general election

('Voters', 'is in', 'Dominican Republic')


sentence:18 year old Max Verstappen becomes youngest ever driver and first Dutchman to win a Formula One race by winning 2016 Spanish Grand Prix

sentence:Tens of thousands of people are evacuated from Old Trafford shortly before kickoff between Manchester United F. C. and A F. C. Bournemouth after a suspect package was found. A controlled explosion was then carried out within stadium. The package is discovered to be an explosives training device left behind earlier in week by a private company following a training exercise involving explosive search dogs

sentence:The Vegetarian written by Han Kang and translated by Deborah Smith wins Man Booker International Prize

sentence:Pfizer Inc announces plans to buy Anacor for dollar 5.2 billion

sentence:At least fifteen students are killed and six are missing af

('124 fahrenheit', 'is in', 'Indian state of Rajasthan')


sentence:At least four National Gendarmerie officers are killed in a helicopter crash in HautesPyrAcnAces in southwestern France

('helicopter crash', 'is in', 'HautesPyrAcnAces')


('helicopter crash', 'is in', 'southwestern France')


sentence:The US Geological Survey USGS reports a magnitude 5.9 earthquake strikes southern part of Northern Territory in central Australia. No damage or injuries are reported

('southern part', 'is in', 'central Australia')


sentence:The Centers for Disease Control and Prevention reports at that least 270 pregnant women in US are infected with Zika virus

('270 pregnant women', 'is in', 'US')


sentence:The Kremlin condemns NATOs membership invitation to Montenegro warning step risks fueling geopolitical tensions in Europe

sentence:NATO foreign ministers meeting at NATO headquarters in Brussels Belgium agree to extend alliances Resolute Support Mission that trains advises and assists Afghanist

('Three policemen', 'are killed in', 'Srinagar India')


('Three policemen', 'are', 'killed')


sentence:The Taliban leadership council reportedly meets to determine a successor to Mullah Akhtar Mansour killed this weekend during an American drone strike in Pakistan

('American drone strike', 'is in', 'Pakistan')


sentence:Germanys Bayer AG makes an unsolicited bid valued at dollar 62 billion cash to takeover American company Monsanto to create worlds largest agricultural chemical company

('unsolicited bid', 'valued at', 'dollar 62 billion')


sentence:An overnight fire at Pitakiatwittaya Christian elementary schools dormitory in northern Chiang Rai Province in Thailand kills eighteen girls between ages of five and twelve. The other 20 girls sleeping in dorm are rescued though five are injured

('northern Chiang Rai Province', 'is in', 'Thailand')


('overnight fire', 'is in', 'northern Chiang Rai Province')


sentence:At least five people are killed after a Cessna 182 lightaircraft 

('Scripps National Spelling Bee', 'is in', 'US')


sentence:Two US Navy FA18s crash in Atlantic Ocean near Outer Banks off coast of US state of North Carolina. The four pilots safely ejected from planes and are being examined in Sentara Norfolk General Hospital in Virginia

('Two US Navy FA18s crash', 'is in', 'Atlantic Ocean')


sentence:At least 20 people drown in southern Mediterranean Sea when a repurposed fishing boat sinks 35 nautical miles north of Zuwara Libya

('people', '20 At', 'least')


sentence:Researchers at United States Department of Defense report in journal Antimicrobial Agents and Chemotherapy discovery in US of a strain of E. coli that includes new gene MCR1 which is resistant to colistin a lastresort antibiotic

('Researchers report', 'is in', 'US of strain of E. coli')


('Researchers report', 'is in', 'journal Antimicrobial Agents')


sentence:Chinese state media Xinhua News Agency criticizes Taiwan President Tsai Ingwen as politically extreme and lacking emotio

In [124]:
event_df

,event_id,date,category,event_title,event_summary,Entity Relation
0,0,2016-01-01,Armed conflicts and attacks,IsraeliE28093Palestinian conflict,A shooting takes place at a pub in Tel Aviv Israel leaving two dead and eight injured. The gunman managed to escape and possibly killed his accomplice an Arab taxi driver whose body was found nearby. The police identified shooter as 30 year old muslim Nisaat Melkham from Arara and searched his home,"(pub, is in, Tel Aviv Israel)"
1,1,2016-01-01,Arts and culture,False,A new 28foot tall statue of Jesus dubbed Jesus de Greatest is unveiled on New Years Day outside St. Aloysius Catholic Church in Abajah village Nigerias Imo state which is described as tallest Jesus statue in Africa,"(tallest Jesus statue, is in, Africa)"
2,2,2016-01-01,Disasters and Incidents,False,About one thousand houses in Manilas Tondo district in Philippines are set ablaze following New Years Eve firecracker festivities that left person dead and 380 others injured,"(one thousand houses, is in, Manilas Tondo district)"
3,3,2016-01-01,International Relations,False,The EUUkraine Free Trade deal officially comes into force coinciding with a Russian food embargo on Ukraine,"(EUUkraine Free Trade, deal officially, comes into force coinciding with Russian food embargo on Ukraine)"
4,4,2016-01-01,Crime and Law,False,The twochild policy takes effect in China allowing couples in country to have at most two children replacing controversial onechild policy. The change in law was announced by ruling Communist Party on October 29 and passed Standing Committee of National Peoples Congress on December 27 five days prior to its effect,"(couples, is in, country)"
...,...,...,...,...,...,...
1995,1995,2016-05-26,Politics and Elections,False,French labor unions led by CGT implement a strike in protest of proposed reforms to labor laws,"(strike, is in, protest of proposed reforms to labor laws)"
1996,1996,2016-05-26,Politics and Elections,2016 United States presidential election,The Associated Press reports that Donald Trump has secured enough delegates to secure Republican Party nomination for President,NaN
1997,1997,2016-05-26,Sports,2016 Stanley Cup playoffs,The Pittsburgh Penguins defeat Tampa Bay Lightning in Game seven of Eastern Conference Final to advance to Stanley Cup Final they will face San Jose Sharks,"(they, face, San Jose Sharks)"
1998,1998,2016-05-27,Armed conflicts and attacks,"Iraqi civil war 2014E28093present , Operation Breaking Terrorism",The US reports 70 militants were killed including commander Maher alBilawi in a Fallujah airstrike on Wednesday,"(70 militants, were, killed including commander Maher alBilawi in Fallujah airstrike on Wednesday)"


In [125]:

for i in range(0,2000):
    event_df.loc[i,'Subject']= np.nan
    event_df.loc[i,'Relation']= np.nan
    event_df.loc[i,'Object']= np.nan



In [126]:
df1=event_df.head(2000)
df1=df1[['event_summary','category','Entity Relation','Subject','Relation','Object']]
df1 = df1[pd.notnull(df1['Entity Relation'])]
df1=df1.reset_index()

In [127]:
df1

,index,event_summary,category,Entity Relation,Subject,Relation,Object
0,0,A shooting takes place at a pub in Tel Aviv Israel leaving two dead and eight injured. The gunman managed to escape and possibly killed his accomplice an Arab taxi driver whose body was found nearby. The police identified shooter as 30 year old muslim Nisaat Melkham from Arara and searched his home,Armed conflicts and attacks,"(pub, is in, Tel Aviv Israel)",nan,nan,nan
1,1,A new 28foot tall statue of Jesus dubbed Jesus de Greatest is unveiled on New Years Day outside St. Aloysius Catholic Church in Abajah village Nigerias Imo state which is described as tallest Jesus statue in Africa,Arts and culture,"(tallest Jesus statue, is in, Africa)",nan,nan,nan
2,2,About one thousand houses in Manilas Tondo district in Philippines are set ablaze following New Years Eve firecracker festivities that left person dead and 380 others injured,Disasters and Incidents,"(one thousand houses, is in, Manilas Tondo district)",nan,nan,nan
3,3,The EUUkraine Free Trade deal officially comes into force coinciding with a Russian food embargo on Ukraine,International Relations,"(EUUkraine Free Trade, deal officially, comes into force coinciding with Russian food embargo on Ukraine)",nan,nan,nan
4,4,The twochild policy takes effect in China allowing couples in country to have at most two children replacing controversial onechild policy. The change in law was announced by ruling Communist Party on October 29 and passed Standing Committee of National Peoples Congress on December 27 five days prior to its effect,Crime and Law,"(couples, is in, country)",nan,nan,nan
...,...,...,...,...,...,...,...
1194,1991,Ken Starr president of Baylor University is stripped of his title after an investigation revealed university mishandled accusations of sexual assault against its football players. The universitys football coach Art Briles is also fired,Crime and Law,"(university, mishandled, accusations of sexual assault against its football players)",nan,nan,nan
1195,1995,French labor unions led by CGT implement a strike in protest of proposed reforms to labor laws,Politics and Elections,"(strike, is in, protest of proposed reforms to labor laws)",nan,nan,nan
1196,1997,The Pittsburgh Penguins defeat Tampa Bay Lightning in Game seven of Eastern Conference Final to advance to Stanley Cup Final they will face San Jose Sharks,Sports,"(they, face, San Jose Sharks)",nan,nan,nan
1197,1998,The US reports 70 militants were killed including commander Maher alBilawi in a Fallujah airstrike on Wednesday,Armed conflicts and attacks,"(70 militants, were, killed including commander Maher alBilawi in Fallujah airstrike on Wednesday)",nan,nan,nan


In [128]:

for j in range(0,1199):
    fo = open('entity', 'w', encoding="utf-8")
    text = df1.loc[j,'Entity Relation']
    type(text)
#     print(text)
    
    for letter in text:
        print (letter)
        fo.write(letter)
        fo.write(";")
#         if letter == '(':
#             print (letter)
#             elif letter == ')':
#                 print (letter)
                
#             else:
#                 
           
    fo.close()        
    with open('entity', 'r',encoding="utf8") as file:
        data = file.read()
#         data = file.read().replace('[', '')
        
#         data = file.read()
        split_data=data.split(';')
        print("Split Data")
        df1.loc[j,'Subject']=split_data[0]
        df1.loc[j,'Relation']=split_data[1]
        df1.loc[j,'Object']=split_data[2]
    
        print(len(split_data))
            
        file.seek(0)
    fo = open('entity', 'w').close()
    print("End")
#     fo.write("")

pub
is in
Tel Aviv Israel
Split Data
4
End
tallest Jesus statue
is in
Africa
Split Data
4
End
one thousand houses
is in
Manilas Tondo district
Split Data
4
End
EUUkraine Free Trade
deal officially
comes into force coinciding with Russian food embargo on Ukraine
Split Data
4
End
couples
is in
country
Split Data
4
End
base
is in
Pathankot Punjab three security force personnel
Split Data
4
End
Gisela Mota Ocampo mayor
is in
Mexicos Morelos state
Split Data
4
End
peaceful rally
is in
support of Dwight
Split Data
4
End
Iraqi Army base Camp Speicher
now known as
Tikrit Air Academy town Tikrit north Baghdad
Split Data
4
End
Indian Air Force base
is in
Pathankot Punjab
Split Data
4
End
residents
prepare for
impact of severe tropical cyclone Ula
Split Data
4
End
temperatures
reach
freezing
Split Data
4
End
its surrounding waters
is in
Atlantic Ocean
Split Data
4
End
attack
is in
Tehran
Split Data
4
End
clash
is with
police
Split Data
4
End
Saudi Arabias execution
is in
Saudi Arabia
Split Data
4

2015
was hottest year since
beginning of meteorological records in 1890
Split Data
4
End
potential new planet Planet Nine
is in
outer Solar System Washington Post
Split Data
4
End
AlShabaab militants attack Lido
is in
Mogadishu
Split Data
4
End
second new case
is in
Sierra Leone
Split Data
4
End
UN Security Council
use of
peacekeeping troops
Split Data
4
End
former commander
is in
Uganda
Split Data
4
End
people
is in
MidAtlantic
Split Data
4
End
school
is in
La Loche Saskatchewan Canada
Split Data
4
End
investigation
breaching
party discipline
Split Data
4
End
Airstrikes
is in
Syrias eastern Deir ezZor Governorate
Split Data
4
End
border
is with
Syria
Split Data
4
End
deaths
ten at
least
Split Data
4
End
demonstration
is in
support of migrants living them
Split Data
4
End
Donald Trump
is in
general election
Split Data
4
End
Poland surveillance law plan
angers
protesters
Split Data
4
End
Blue Origins rocket New Shepard
has
has reflown
Split Data
4
End
Heavy snowfall
is in
western Japan


Albert Pahimi PadackAc
is
appointed
Split Data
4
End
US Supreme Court Justice Antonin Scalia
dies at
age 79
Split Data
4
End
Christchurch
is in
New Zealand
Split Data
4
End
earthquake death toll
rises to
one hundred
Split Data
4
End
New Hampshire subzero temperatures
is in
White Mountains
Split Data
4
End
second round
is in
presidential election
Split Data
4
End
Jocelerme Privert
pending elections at_time
later year
Split Data
4
End
checkpoint
is in
Russias republic of Dagestan
Split Data
4
End
Crossborder
shelling from
Yemen
Split Data
4
End
Ed Sheeran
wins Grammy Award for
Song of Year
Split Data
4
End
chemical plants
were close to
too areas
Split Data
4
End
Three South Korean army soldiers
die after
Bell UH1 Iroquois helicopter crashes in city of Chuncheon Gangwon Province
Split Data
4
End
Ukraine
bans
Russian cargo trucks
Split Data
4
End
Herzegovina
join
European Union
Split Data
4
End
twoday summit
is in
Rancho Mirage California
Split Data
4
End
Two German intelligence officers
g

End
border
is with
Greece
Split Data
4
End
Pakistani authorities
hang Mumtaz Qadri for
his role in murder of reformist Punjab governor Salmaan Taseer in 2011
Split Data
4
End
stable condition
is in
Madison Local School cafeteria
Split Data
4
End
Democratic Party front runner
is in
race
Split Data
4
End
raid
is in
northern Iraq
Split Data
4
End
bus
is in
northwestern Oman
Split Data
4
End
50 homes
is in
Ripley New York
Split Data
4
End
first Zika virus case
is confirmed in
US state of New Hampshire
Split Data
4
End
grand jury
is in
US state of Pennsylvania
Split Data
4
End
their gender
is in
state public schools
Split Data
4
End
noconfidence vote
is in
South African Parliament
Split Data
4
End
March 22 2016
is in
Havana
Split Data
4
End
Chineseinvested company
is in
Laos official Xinhua News Agency reports
Split Data
4
End
his home
is in
occupied West Bank
Split Data
4
End
cell
is in
city of Irbid
Split Data
4
End
attack
is In
Jalalabad Afghanistan
Split Data
4
End
Mining company Samarc

4
End
security checkpoint
is in
city of Arish North Sinai Governorate
Split Data
4
End
landing
is in
Russian city of RostovonDon
Split Data
4
End
18 others
are rescued in
Gulf of Mexico 130 miles west of Marco Island Florida
Split Data
4
End
resignation
impeachment of
President Dilma Rousseff
Split Data
4
End
Donald Trumps campaign rally
is with
former Arizona Governor Jan Brewer
Split Data
4
End
weekends suicide bombing
is in
Istanbul
Split Data
4
End
current President Thomas Boni Yayi
completing
two terms
Split Data
4
End
Voters
is in
Cape Verde
Split Data
4
End
Voting
is
underway
Split Data
4
End
Voters
is in
Republic of Congo
Split Data
4
End
underway
is in
Senegal
Split Data
4
End
bakery
is in
Belgrade Serbia
Split Data
4
End
converted hotel
is in
Mali capital Bamako
Split Data
4
End
he
made about
man
Split Data
4
End
International Criminal Court
finds
exVice President of Democratic Republic of Congo JeanPierre Bemba guilty of war crimes
Split Data
4
End
Myanmars new parliament
is

4
End
He
helped
doctor budget accounting
Split Data
4
End
people
two at
least
Split Data
4
End
people
28 at
least
Split Data
4
End
indefinite halt
is in
war crimes trial of Croatian Serb rebel leader Goran HadiA
Split Data
4
End
approval
is with
61.1 percent reaching for referendum valid
Split Data
4
End
UNbacked Libyan Presidential Council
is in
Tripoli
Split Data
4
End
Beninese businessman Patrice Talon
is sworn in as
new President of Benin
Split Data
4
End
projectiles
hit
Turkeys Kilis Province
Split Data
4
End
Trent Harmon
is
declared
Split Data
4
End
Hospitality
is in
response to potential backlash from Mississippi Governor Phil Bryants signing of knownas religious freedom bill
Split Data
4
End
Whyalla Steelworks
is in
South Australia
Split Data
4
End
Vanuatu third earthquake
is in
same region
Split Data
4
End
it
sought because of
genetic abnormalities
Split Data
4
End
Hashim ThaAsi
is sworn in as
new President of Kosovo
Split Data
4
End
transitional government
is with
President S

4
End
deal
is with
US government
Split Data
4
End
proper procedures
were
followed
Split Data
4
End
early summer heatwave
is in
India
Split Data
4
End
elevated bike lane
is in
Rio de Janeiro Brazil
Split Data
4
End
their ties
is with
Philippines
Split Data
4
End
Britons
keep UK in
European Union
Split Data
4
End
two plane
powered by
only sun
Split Data
4
End
Olympic flame
is
lit outside Temple of Hera in Olympia Greece by Greek actress Katerina Lehou
Split Data
4
End
Pittsburgh Pirates baseball players
being
exposed
Split Data
4
End
Shiite mosque
is in
southwestern Baghdad
Split Data
4
End
wounded eleven others
is in
airstrikes between September ten 2015
Split Data
4
End
Eurovision Song Contest
is in
Stockholm Sweden
Split Data
4
End
Carver County Minnesota Sheriffs office
determine
cause of death
Split Data
4
End
creditors
pursue
its owner Australian politician Clive Palmer
Split Data
4
End
problems
is with
gearshift confusing drivers
Split Data
4
End
landslide
is in
Tawang District
Sp

4
End
military installation
is in
Turkey
Split Data
4
End
civilians
sixteen at
least
Split Data
4
End
truce
is with
Israeli military
Split Data
4
End
53
injured after
bus skids off bridge
Split Data
4
End
Ohio Governor John Kasich
suspends
his campaign
Split Data
4
End
its researchers
have developed in
Yorktown Heights New York
Split Data
4
End
May 28
is in
Milan Italy
Split Data
4
End
many
are wounded in
twin blasts in central Syria
Split Data
4
End
Philippine peso
fell
zero
Split Data
4
End
ravine
is in
Bolivias eastern Santa Cruz province
Split Data
4
End
California court
finds
Grim Sleeper serial killer guilty
Split Data
4
End
Users
make
password changes
Split Data
4
End
conflict
is with
President Recep Tayyip ErdoAŸan
Split Data
4
End
AlNusra Front
is in
Khan Tuman south of Aleppo Syria
Split Data
4
End
detention centre
be
closed
Split Data
4
End
Sadiq Khan
succeed Boris Johnson as
Mayor of London Khan
Split Data
4
End
assailants
is in
insider attack
Split Data
4
End
soldiers
thir

4
End
Chinese state media Xinhua News Agency
criticizes
Taiwan President Tsai Ingwen
Split Data
4
End
child sexual abuse
punishable by
chemical castration
Split Data
4
End
university
mishandled
accusations of sexual assault against its football players
Split Data
4
End
strike
is in
protest of proposed reforms to labor laws
Split Data
4
End
they
face
San Jose Sharks
Split Data
4
End
70 militants
were
killed including commander Maher alBilawi in Fallujah airstrike on Wednesday
Split Data
4
End
people
two at
least
Split Data
4
End


In [130]:
for i in range(0,1199):
    print( '{' + '"'+ df1['Subject'][i]+'"'+','+ '"'+df1['Relation'][i]+ '"'+','+ '"'+df1['Object'][i]+ '"'+ '}'+',')

{"pub","is in","Tel Aviv Israel"},
{"tallest Jesus statue","is in","Africa"},
{"one thousand houses","is in","Manilas Tondo district"},
{"EUUkraine Free Trade","deal officially","comes into force coinciding with Russian food embargo on Ukraine"},
{"couples","is in","country"},
{"base","is in","Pathankot Punjab three security force personnel"},
{"Gisela Mota Ocampo mayor","is in","Mexicos Morelos state"},
{"peaceful rally","is in","support of Dwight"},
{"Iraqi Army base Camp Speicher","now known as","Tikrit Air Academy town Tikrit north Baghdad"},
{"Indian Air Force base","is in","Pathankot Punjab"},
{"residents","prepare for","impact of severe tropical cyclone Ula"},
{"temperatures","reach","freezing"},
{"its surrounding waters","is in","Atlantic Ocean"},
{"attack","is in","Tehran"},
{"clash","is with","police"},
{"Saudi Arabias execution","is in","Saudi Arabia"},
{"Indian Air Force base","is in","Pathankot Punjab"},
{"Islamic State","is in","Libya attacks Libyan oil Port of sidra"},
{

In [133]:
df1

,index,event_summary,category,Entity Relation,Subject,Relation,Object
0,0,A shooting takes place at a pub in Tel Aviv Israel leaving two dead and eight injured. The gunman managed to escape and possibly killed his accomplice an Arab taxi driver whose body was found nearby. The police identified shooter as 30 year old muslim Nisaat Melkham from Arara and searched his home,Armed conflicts and attacks,"(pub, is in, Tel Aviv Israel)",pub,is in,Tel Aviv Israel
1,1,A new 28foot tall statue of Jesus dubbed Jesus de Greatest is unveiled on New Years Day outside St. Aloysius Catholic Church in Abajah village Nigerias Imo state which is described as tallest Jesus statue in Africa,Arts and culture,"(tallest Jesus statue, is in, Africa)",tallest Jesus statue,is in,Africa
2,2,About one thousand houses in Manilas Tondo district in Philippines are set ablaze following New Years Eve firecracker festivities that left person dead and 380 others injured,Disasters and Incidents,"(one thousand houses, is in, Manilas Tondo district)",one thousand houses,is in,Manilas Tondo district
3,3,The EUUkraine Free Trade deal officially comes into force coinciding with a Russian food embargo on Ukraine,International Relations,"(EUUkraine Free Trade, deal officially, comes into force coinciding with Russian food embargo on Ukraine)",EUUkraine Free Trade,deal officially,comes into force coinciding with Russian food embargo on Ukraine
4,4,The twochild policy takes effect in China allowing couples in country to have at most two children replacing controversial onechild policy. The change in law was announced by ruling Communist Party on October 29 and passed Standing Committee of National Peoples Congress on December 27 five days prior to its effect,Crime and Law,"(couples, is in, country)",couples,is in,country
...,...,...,...,...,...,...,...
1194,1991,Ken Starr president of Baylor University is stripped of his title after an investigation revealed university mishandled accusations of sexual assault against its football players. The universitys football coach Art Briles is also fired,Crime and Law,"(university, mishandled, accusations of sexual assault against its football players)",university,mishandled,accusations of sexual assault against its football players
1195,1995,French labor unions led by CGT implement a strike in protest of proposed reforms to labor laws,Politics and Elections,"(strike, is in, protest of proposed reforms to labor laws)",strike,is in,protest of proposed reforms to labor laws
1196,1997,The Pittsburgh Penguins defeat Tampa Bay Lightning in Game seven of Eastern Conference Final to advance to Stanley Cup Final they will face San Jose Sharks,Sports,"(they, face, San Jose Sharks)",they,face,San Jose Sharks
1197,1998,The US reports 70 militants were killed including commander Maher alBilawi in a Fallujah airstrike on Wednesday,Armed conflicts and attacks,"(70 militants, were, killed including commander Maher alBilawi in Fallujah airstrike on Wednesday)",70 militants,were,killed including commander Maher alBilawi in Fallujah airstrike on Wednesday


In [132]:
# for i in range(0,66):
#     df1.loc[i,'Subject']=df1.loc[i,'Subject'].replace('TelAviv','Tel Aviv')
#     df1.loc[i,'Object']=df1.loc[i,'Object'].replace('TelAviv','Tel Aviv')

In [ ]:
for i in range(0,66):
    nere=sNLP.ner(df1.loc[i,'Object'])
    print(nere)

In [135]:
for i in range(0,1199):
    print( '{' 'subject:'+ '"ex:'+ "News Current Events"+'"'+',     '+'predicate:'+'"rdf:type'+ '"'+',    '+'object:'+ '"ex:'+df1['category'][i]+ '"'+ '}'+',')
    print( '{' 'subject:'+ '"ex:'+ df1['category'][i]+'"'+',     '+'predicate:'+'"rdf:type'+ '"'+',    '+'object:'+ '"ex:'+df1['Subject'][i]+ '"'+ '}'+',')
    print( '{' 'subject:'+ '"ex:'+ df1['Subject'][i]+'"'+',     '+'predicate:'+'"ex:'+df1['Relation'][i]+ '"'+',    '+'object:'+ '"ex:'+df1['Object'][i]+ '"'+ '}'+',')
#     print( '{' 'subject:'+ '"ex:'+ df1['Object'][i]+'"'+',     '+'predicate:'+'"ex:'+df1['Relation'][i]+ '"'+',    '+'object:'+ '"ex:'+df1['Subject'][i]+ '"'+ '}'+',')

{subject:"ex:News Current Events",     predicate:"rdf:type",    object:"ex:Armed conflicts and attacks"},
{subject:"ex:Armed conflicts and attacks",     predicate:"rdf:type",    object:"ex:pub"},
{subject:"ex:pub",     predicate:"ex:is in",    object:"ex:Tel Aviv Israel"},
{subject:"ex:News Current Events",     predicate:"rdf:type",    object:"ex:Arts and culture"},
{subject:"ex:Arts and culture",     predicate:"rdf:type",    object:"ex:tallest Jesus statue"},
{subject:"ex:tallest Jesus statue",     predicate:"ex:is in",    object:"ex:Africa"},
{subject:"ex:News Current Events",     predicate:"rdf:type",    object:"ex:Disasters and Incidents"},
{subject:"ex:Disasters and Incidents",     predicate:"rdf:type",    object:"ex:one thousand houses"},
{subject:"ex:one thousand houses",     predicate:"ex:is in",    object:"ex:Manilas Tondo district"},
{subject:"ex:News Current Events",     predicate:"rdf:type",    object:"ex:International Relations"},
{subject:"ex:International Relations",    

{subject:"ex:News Current Events",     predicate:"rdf:type",    object:"ex:Politics and Elections"},
{subject:"ex:Politics and Elections",     predicate:"rdf:type",    object:"ex:cuts"},
{subject:"ex:cuts",     predicate:"ex:is in",    object:"ex:disability benefit proposed by George Osborne Chancellor of Exchequer"},
{subject:"ex:News Current Events",     predicate:"rdf:type",    object:"ex:Armed conflicts and attacks"},
{subject:"ex:Armed conflicts and attacks",     predicate:"rdf:type",    object:"ex:Airstrikes"},
{subject:"ex:Airstrikes",     predicate:"ex:is in",    object:"ex:northern syria"},
{subject:"ex:News Current Events",     predicate:"rdf:type",    object:"ex:Armed conflicts and attacks"},
{subject:"ex:Armed conflicts and attacks",     predicate:"rdf:type",    object:"ex:security checkpoint"},
{subject:"ex:security checkpoint",     predicate:"ex:is in",    object:"ex:city of Arish North Sinai Governorate"},
{subject:"ex:News Current Events",     predicate:"rdf:type",    o

{subject:"ex:News Current Events",     predicate:"rdf:type",    object:"ex:Crime and Law"},
{subject:"ex:Crime and Law",     predicate:"rdf:type",    object:"ex:concert"},
{subject:"ex:concert",     predicate:"ex:is in",    object:"ex:Austrian town Nenzing"},
{subject:"ex:News Current Events",     predicate:"rdf:type",    object:"ex:Crime and Law"},
{subject:"ex:Crime and Law",     predicate:"rdf:type",    object:"ex:two nightclubs"},
{subject:"ex:two nightclubs",     predicate:"ex:is in",    object:"ex:Mexican state of Veracruz"},
{subject:"ex:News Current Events",     predicate:"rdf:type",    object:"ex:Politics and Elections"},
{subject:"ex:Politics and Elections",     predicate:"rdf:type",    object:"ex:Voters"},
{subject:"ex:Voters",     predicate:"ex:is in",    object:"ex:Vietnam"},
{subject:"ex:News Current Events",     predicate:"rdf:type",    object:"ex:Politics and Elections"},
{subject:"ex:Politics and Elections",     predicate:"rdf:type",    object:"ex:Voters"},
{subject:"e